### Implementation
* Move to Python3
* Git hub update
* Load Pierre protein list
* Check for exisiting path between any node and a seed node after edge trimming
* Remove network not connected with seed node 

### Study
$\Delta$ SimPct treshold

 * How many networks
 * Tabulate individual protein degree across the generated networks

| Protein| 80% | 60% | ... | 10% |
| ------ | --- | --- | --- | --- |
| XXX    | 22  | 10  | ... | NA  |



GO analysis:
* Global tablechart of GO terms occurences across the network
* For given Protein and its 1st neighbors, display the GO annotations distribution of the proteins and their templates

Note montage necessaire

    sshfs glaunay@arwen://mobi/group/divisome /Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen  -o umask=0002,allow_other,defer_permissions,IdentityFile=~/.ssh/id_rsa
    
Example BLAST utilisé
```shell
gunzip -c ~/work/projects/pneumocoque/JM_Rebuild/arwen/2017_10_04_DUPLICATE_RESULTS/INTACT_VERSUS_R6R/BLAST_OUTPUT/P0/P0CI74/blast.out.gz | less
```

Gestion des IDs de Intact
Travail sur un subset (head 200)
> /home/lbarlet/WorkOnIntact/subset_intact.txt

split pour récup deux première colonnes 
ensuite trie pour savoir quelle ID est la plus présentes,
la seconde, ...

/!\ la deuxième ne doit pas présenter la première, etc /!\


TODO 

  1. Récuperer interaction experiemntals suplémentaires
  2. Alignement de séquences sur protéines du divisome
  3. Définition de blocs
  4. Métrique homologie basée sur la similarité dans ces blocs.



In [9]:
import sys
import os
import subprocess
sys.path.append('/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/omegaLoMo/lib')
import core as ca
import createTopo as cT
#import graph as graph
import smallWork as sW
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%matplotlib inline
import matplotlib.mlab as mlab
import matplotlib.patches as mpatches
from collections import Counter
from collections import defaultdict
import fnmatch
import networkx as nx
import matplotlib.pyplot as plt
import copy
import collections
import numpy as np
import operator
import json
import math
%autoreload 2

* MITAB_file and filterIds can be filled when reading serialized data, or read from scratch, using input files.
* MITAB_file contains IntAct informations, restricted to physical interactions. 
It is a subset of the informations
from /mobi/group/divisome/physical_intact.mitab.txt, after serialization by an external python script.
* filterIds contains information about S pneumoniae
* intactR6filtered_topology is a dictionary, with pairs of UniProt Ids of proteins from IntAct interactions, that match with R6 sequences,
non redundant.

In [11]:
%autoreload 2
# Creating an IntAct topolgy dictionary from scratch
# all physical interactions from IntAct: 348,806 interactions 
MITAB_file="/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/physical_intact.mitab.txt"
# a subset of 20,000 interactions 
MITAB_file="/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/physical_intact.mitab_20000.txt"
filterIds="/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/anaBlast/allInR6Test2"

# same, but sourcing from the directory generated by JM
#MITAB_file="/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/2017_10_04_DUPLICATE_RESULTS/INTACT_VERSUS_R6R/physical_intact.mitab_20000.txt"
MITAB_file="/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/2017_10_04_DUPLICATE_RESULTS/INTACT_VERSUS_R6R/physical_intact.mitab.txt"
# So far, we only parse the IntAct ids, not the R6 info
filterIds="/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/2017_10_04_DUPLICATE_RESULTS/POST_PROCESS_INTACT_VERSUS_R6R/allHitsInR6.txt"
# 

# create Topolgy object
topo = cT.Topology()
# this step, with the complete MITAB_file physical_intact.mitab.txt, takes
# about 40 minutes 
# parse MITAB file
#topo.parseIntactMitab(MITAB_file)
# filter the IntAct interactions, to remove protein Ids with no match with R6
#topo.filterWith(filterIds)

# Create an IntAct topology from object deserialisation
topo.deSerialize('/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/serializedData/newDic.json')

intactR6filtered_topology=topo.newDic
#print intactR6filtered_topology

# OK 

## File format of /mobi/group/divisome/anaBlast/allInR6Test2
** File content: **
Q6A162\n

Q8DRI0:74,316,76,31,1.63488e-07

Q8DPJ9:233,491,101,47,8.67392e-07

Q8DN05:124,460,108,32,0.0420328   

** Explanation **

Means that the IntAct UniprotId Q6A162 has 3 hits in the R6 proteome: Q8DRI0, Q8DPJ9, and Q8DN05. 
    Informations after the ":" 
    are the alignment statistics : Hsp_hit-from, Hsp_hit-to, Hsp_positive, Hsp_identity, Hsp_evalue
    /mobi/group/divisome/UsineBlastArrayIntact_vs_R6/IntactVsR6/swork/Q6/Q6A162/logs

** Native BLAST results **
They are stored here :
/mobi/group/divisome/UsineBlastArrayIntact_vs_R6/IntactVsR6/swork/
        
The folder organization is such that for the IntAct sequence Q6A162 :
/mobi/group/divisome/UsineBlastArrayIntact_vs_R6/IntactVsR6/swork/Q6/Q6A162/logs
    
** Generation of BLAST results **

See /mobi/group/divisome/UsineBlastArrayIntact_vs_R6/main_array.sh
and /mobi/group/divisome/UsineBlastArrayIntact_vs_R6/job4one.sh




## Omega Set creation
The omegaSet object stores query organism homologs vector for each protein found in PSICQUIC

#### From scratch, omegaSet object construction
  + From Loic's files 
    ```python
    indexR6 = '/Volumes/arwen/mobi/group/divisome/serializedData/indexR6'
    blastResults='/Volumes/arwen/mobi/group/divisome/UsineBlastArrayIntact_vs_R6/IntactVsR6/swork/'
    ```
  + From Juliettes rebuildt files 
    ```python
    indexR6 = '/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/2017_10_04_DUPLICATE_RESULTS/POST_PROCESS_INTACT_VERSUS_R6R/R6_index.txt'
    blastResults='/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/2017_10_04_DUPLICATE_RESULTS/POST_PROCESS_INTACT_VERSUS_R6R/OUTPUT/'
    ```
  
  + Calling constructor on XML **OR** JSON ressources
  ```python
  omegaSet=ca.HomegaSet(queryIdList=indexR6,blastXmlPath=blastResults)
  omegaSet=ca.HomegaSet(queryIdList=indexR6,jsonSerialPath=blastResults)
  ```

#### Deserializing a previous instance of _omegaSet_
  + Calling constructor on serial JSON ressource   
    ```python
    omegaSet=ca.HomegaSet(queryIdList=indexR6,bean="/Users/guillaumelaunay/grosTas.json")
    ```
  
#### Serializing current instance of _omegaSet_
 + Calling method with path to file for writing   
    ```python
    omegaSet.serialize("/Users/guillaumelaunay/grosTas.json")
    ```
    

In [4]:
indexR6 = '/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/2017_10_04_DUPLICATE_RESULTS/POST_PROCESS_INTACT_VERSUS_R6R/R6_index.txt'
blastResults='/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/2017_10_04_DUPLICATE_RESULTS/POST_PROCESS_INTACT_VERSUS_R6R/OUTPUT/'
omegaSet=ca.HomegaSet(queryIdList=indexR6, bean="/Users/guillaumelaunay/grosTas.json")



<span style="color:brown;font-weight:bold"> GL -- Needs documentation reformat </span>

## Omega Set description

OmegaSet is a list of omegaVectors objects.
    
** Description of the omegaVector object **

* a dictionary with a key "template ID", value= a UniprotId from IntAct,
    * a key "Pairs Data", value= list of dictionnaries 
        * "query" : R6 Ids
        * "param" : list of HSP statistics
        * "template" : the template Id (identical to template ID)
            
** FullOmegaSet_coverage.json **
a serialized file with all HSP statistics, for all IntAct proteins.
Hence, we need to use the argument queryIdList=indexR6, to filter out HSPs that do not belong to R6.
    
** Generation of indexR6 and FullOmegaSet_coverage.json **
* indexR6: a simple list of Uniprot Ids from the R6 proteome
    
    ```bash
    head  /Volumes/arwen/mobi/group/divisome/serializedData/indexR6
    ```
    
````
sp|P0A2U7|ADCC|STRR6.seq
sp|P0A2U9|AMIE|STRR6.seq
sp|P0A2V9|PSTB3|STRR6.seq
sp|P0A2W1|ACP|STRR6.seq
sp|P0A2W7|ACPS|STRR6.seq
sp|P0A2W9|ALR|STRR6.seq
sp|P0A2X4|EXOA|STRR6.seq
sp|P0A2Y3|ARGR|STRR6.seq
sp|P0A2Y7|AROC|STRR6.seq
sp|P0A2Y9|ATP6|STRR6.seq
```
* FullOmegaSet_coverage.json   



## From omega Set to interactome

#### 1. Connect omegaVector objects
```python
omegaMatrix = ca.OmegaMatrix(topo = intactR6filtered_topology, omegaSet = omegaSet)
omegaMatrix.reduceAndVectorInject()
```
#### 2. Build the adjacency matrix in R6
```python
queryTopo = omegaMatrix.project()
```

##### Remarks
list of predicted interactions in R6. Each R6 interaction (= pair of nodes) is associated with 1 (or more) IntAct pairs, and their homology relationships are characterized by HSP statistics, ofc one predicted pair can be supported by several IntAct pairs


### Homology information support description
<figure>
<img src="binary.jpg", width="250px">
<figcaption>
Any interaction is binary, one node is arbitrarly defined as the "loQuery" (uniprotID: P60414) and the other as the "hiQuery" (uniprotID: Q8DPU0).
</figcaption>
</figure>



##### edgeAttributes tuple
It is a 3 elements tuple structured as :

```python
(Q8DR57, #Node uniprot Identifier on the "lowQuery" side of the edge
 Q9EUQ7, #Node uniprot Identifier on the "highQuery" side of the edge
 {'lowQueryParam': [
     # Each homolog relationship supporting interaction inference on the "low" side of the edge
     [u'1', u'105', u'66', u'49', u'3.02566e-17', u'98', u'3', u'98', u'109', 'P0CI74']
     ],
  'highQueryParam': [
      # Each homolog relationship supporting interaction inference on the "high" side of the edge
      [u'3', u'230', u'138', u'84', u'7.50339e-38', u'251', u'6', u'242', u'242', 'P35154']
      ]
})
```
##### Content of the Interaction inference Parameters Array

<span style="color:red;font-weight:bold">NB:</span>
*When refering to the blast output file hit and query are reversed, ie: query sequences come from PSICQUIC and target/template from R6*. <span style="color:red">LoMo messed it up, hence the cluncky following ordering of **query/template** sequence lengths.

In the following document **Query** refers to bacterial proteome element and **Template** to PSICQUIC element.

<ul style="list-style-type: none;">
  <li><span style='color:steelblue;font-weight:bold'>[1]</span>  Hsp position start in **query** sequence</li>
  <li><span style='color:steelblue;font-weight:bold'>[2]</span>  Hsp position stop in **query** sequence</li>
  <li><span style='color:steelblue;font-weight:bold'>[3]</span>  Hsp alignemnt positive match</li>
  <li><span style='color:steelblue;font-weight:bold'>[4]</span>  Hsp alignemnt identical match</li>
  <li><span style='color:steelblue;font-weight:bold'>[5]</span>  Hsp alignment eValue</li>
  <li><span style='color:steelblue;font-weight:bold'>[6]</span>  **Template** sequence total length</li>
  <li><span style='color:steelblue;font-weight:bold'>[7]</span>  Hsp position start in **template** sequence</li>
  <li><span style='color:steelblue;font-weight:bold'>[8]</span>  Hsp position stop in **template** sequence</li>
  <li><span style='color:steelblue;font-weight:bold'>[9]</span>  **Query** sequence length</li>
  <li><span style='color:steelblue;font-weight:bold'>[10]</span> **Template** uniprot identifier</li>
  
</ul>


eg:

The following XML blast output (`P0CI74/blast.out.gz`) will produce previous egde homology support

```xml
 <BlastOutput_query-len>98</BlastOutput_query-len>
 ... [LAST PSI-BLAST ITERATION]
 <Hit>
          <Hit_num>4</Hit_num>
          <Hit_id>sp|Q8DR57|GPSB_STRR6</Hit_id>
          <Hit_def>Cell cycle protein GpsB OS=Streptococcus pneumoniae (strain ATCC BAA-255 / R6) GN=gpsB PE=3 SV=2</Hit_def>
          <Hit_accession>Q8DR57</Hit_accession>
          <Hit_len>109</Hit_len>
          <Hit_hsps>
            <Hsp>
              <Hsp_num>1</Hsp_num>
              <Hsp_bit-score>87.5355</Hsp_bit-score>
              <Hsp_score>216</Hsp_score>
              <Hsp_evalue>3.02566e-17</Hsp_evalue>
              <Hsp_query-from>3</Hsp_query-from>
              <Hsp_query-to>98</Hsp_query-to>
              <Hsp_hit-from>1</Hsp_hit-from>
              <Hsp_hit-to>105</Hsp_hit-to>
              <Hsp_query-frame>1</Hsp_query-frame>
              <Hsp_hit-frame>1</Hsp_hit-frame>
              <Hsp_identity>49</Hsp_identity>
              <Hsp_positive>66</Hsp_positive>
              <Hsp_gaps>9</Hsp_gaps>
              <Hsp_align-len>105</Hsp_align-len>
              <Hsp_qseq>ADKVKLSAKEILEKEFKTGVRGYKQEDVDKFLDMIIKDYETFHQEIEELQQENLQLKKQLEEASKKQPVQ---------SNTTNFDILKRLSNLEKHVFGSKLYD</Hsp_qseq>
              <Hsp_hseq>MASIIFSAKDIFEQEFGREVRGYNKVEVDEFLDDVIKDYETYAALVKSLRQEIADLKEELTRKPKPSPVQAEPLEAAITSSMTNFDILKRLNRLEKEVFGKQILD</Hsp_hseq>
              <Hsp_midline>   +  SAK+I E+EF   VRGY + +VD+FLD +IKDYET+   ++ L+QE   LK++L    K  PVQ         S+ TNFDILKRL+ LEK VFG ++ D</Hsp_midline>
            </Hsp>
          </Hit_hsps>
        </Hit>
```

In [12]:
omegaMatrix = ca.OmegaMatrix(topo = intactR6filtered_topology, omegaSet = omegaSet)
omegaMatrix.reduceAndVectorInject()
queryTopo = omegaMatrix.project()

## Graphs Functions

Utility functions to 
  + Load specific seed datasets
    ```python
    seedList = loadSeedList('all' OR 'phino' OR 'massida')
    ```
  + dump network topology under json format compliant with [Dforce,D3--style](https://bl.ocks.org/vasturiano/02affe306ce445e423f992faeea13521)
   ```python
    networkAsD3_dict = nEncode(G)
    with open('interactome.json', 'w') as outfile:
        json.dump(networkAsD3_dict, outfile)
    ```

In [14]:
import pandas
import json

def loadSeedList(lType='all'):
    fileMap = {
        'phino' : '/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/DivAlreadyKnow_litt_phino.txt',
        'massida' : '/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/DivAlreadyKnow_litt_massida.txt',
        'barletMerge': '/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/serializedData/divKnownR6.txt',
        'garcia' : '/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/pierre_uniprot.tsv',
        'all' : '/Users/guillaumelaunay/work/projects/pneumocoque/JM_Rebuild/arwen/current_merge.txt'
    }
    if lType not in fileMap:
        raise IndexError("no set named " + lType)
        
    pFrame = pandas.read_csv(fileMap[lType], sep='\t', lineterminator='\n')
    return [ uniprotID for uniprotID in pFrame.loc[:]["Entry"] ]

def nEncode(G):
    data = { 'nodes' : [], 'links' : [] }
    
    for e in G.edges(data=True):
        data['links'].append({ 'source' : str(e[0]), 'target' : str(e[1]), 'value' : len(e[2]['lowQueryParam']),
                             'data' : e[2] })

    for n in G.nodes(data=True):
        data['nodes'].append({'id' : str(n[0]), 'group' : n[1]['group'], 'val' : G.degree(n[0]) })
    return data

In [15]:
import graph as graph
iObj2 = graph.Interactome(queryTopo)
#init = iOBj.Gtotal

In [16]:
seedList = loadSeedList()
#seedList=['Q8DNV9']
G = iObj2.graph(seedNodes=seedList[:], simPct=60.0)

# RESUME HERE
#nx.draw(G,pos=nx.spring_layout(G)) # use spring layout

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
Total number of pruned edges is 89567 / 90266


In [18]:
graph.NodeView(G)

P0A336,30
Q8DPS0,28
P59661,24
Q8CWN9,21
P63788,20
P66041,17
Q8DPM2,16
Q8DRL7,16
Q8DPM9,15
Q8DRB3,15
P0A3R0,14


In [14]:
with open('/Users/guillaumelaunay/work/projects/pneumocoque/3Dnetwork/3D_force/networks/interactome_JM_test.json', 'w') as outfile:
    json.dump(nEncode(G), outfile)

In [ ]:
dir(queryTopo)

In [15]:
for e in G.edges(data=True):
    print e

(Q8DR57, Q9EUQ7, {'lowQueryParam': [[u'1', u'105', u'66', u'49', u'3.02566e-17', u'98', u'3', u'98', u'109', 'P0CI74']], 'highQueryParam': [[u'3', u'230', u'138', u'84', u'7.50339e-38', u'251', u'6', u'242', u'242', 'P35154']]})
(Q8DPK7, Q8DPK7, {'lowQueryParam': [[u'10', u'661', u'464', u'331', u'2.2889e-113', u'657', u'2', u'654', u'662', 'Q9PPM7']], 'highQueryParam': [[u'10', u'661', u'464', u'331', u'2.2889e-113', u'657', u'2', u'654', u'662', 'Q9PPM7']]})
(Q8DNA5, Q8CWM8, {'lowQueryParam': [[u'1', u'241', u'166', u'118', u'6.48918e-64', u'241', u'1', u'241', u'259', 'P0A7V0']], 'highQueryParam': [[u'24', u'890', u'548', u'391', u'1.88184e-108', u'891', u'6', u'873', u'890', 'P0A9Q7']]})
(Q8DNR6, Q8CWP9, {'lowQueryParam': [[u'1', u'175', u'107', u'60', u'1.38121e-20', u'164', u'1', u'164', u'262', 'P71021']], 'highQueryParam': [[u'1', u'798', u'509', u'365', u'1.36775e-137', u'805', u'1', u'805', u'798', 'P94461']]})
(Q9EUQ7, Q9EUQ7, {'lowQueryParam': [[u'3', u'230', u'138', u'84',